# TP5 - Ejercicio 1

Se desea digitalizar una señal cuyo espectro se muestra a continuación, trazado en azul.

![ej1_consigna](img/ej1_consigna.JPG)

El rango de frecuencias de interés se centra entre DC y $10 kHz$, mientras que las frecuencias superiores a esta no aportan información.

Se sabe que el sistema digital introduce un ruido de densidad espectral constante, que se encuentra en $60 dB$ por debajo del nivel máximo de la señal.

Este ruido es consecuencia de los errores de cuantificación y del ruido propio de la electrónica del conversor, y no puede ser eliminado. El mismo está representado en el trazo rojo.

A partir de lo anterior, se pide:

A) Definir un valor para la frecuencia de muestreo $f_s$ del ADC, justificando su respuesta.

B) Diseñar un filtro analógico pasa bajo anti alias, esto es, que atenúe las componentes frecuenciales ubicadas por encima de la frecuencia de Nyquist, para evitar efecto alias. Explicar el criterio utilizado para fijar la atenuación en la banda detenida.

C) Repetir el punto b), pero eligiendo una $f_s$ diez veces más grande a la elegida en a). Comparar ambos filtros anti alias y evaluar las ventajas/desventajas de esta modificación.

D) Repetir el punto b), pero considerando que se requiere una fase aprox. lineal hasta $10 kHz$.


## Resolución

A) Para poder evitar el efecto del aliasing, deberíamos elegir una frecuencia de muestreo que sea al menos 2 veces mayor que la máxima frecuencia de interés. Considerando que nuestro rango de interés llega hasta los $10 KHz$, deberíamos elegir:

$f_s \geq 2 . f_{max}$

$f_s \geq 20KHz$

Seleccionamos una $f_s = 30KHz$ para tener un margen en el sistema y no estar en la condición crítica.

B) La señal que se quiere digitalizar, en $f=10KHz$ se encuentra $40dB$ por debajo de su valor de potencia máximo. Considerando que el piso de ruido digital se encuentra en $-60dB$, deberíamos diseñar un filtro que tenga una atenuación mínima de $20dB$ para todas las frecuencias superiores a $f_{nyq} = \frac{f_s}{2} = 15KHz$.

Teniendo esto en consideración podemos plantear la siguiente plantilla:

<div><img src="img/ej1_plantilla_desnormalizada.jpeg" width="500"/></div>

Al normalizarla queda:

<div><img src="img/ej1_plantilla_normalizada.jpeg" width="500"/></div>

Se puede preveer que requeriremos un rolloff "rápido", por lo cual planteamos una aproximación de Chebyshev:

$|T(j\omega)| = \frac{1}{1+\varepsilon^2.C_n^2}$

Utilizamos python para averiguar $\varepsilon$ y el orden $n$ del filtro.

In [2]:
import numpy as np

ws = 1.5
ee = 10**(1/10)-1
print(f'ee: {ee}')

for n in range(1,6):
    Cn = np.cosh(n*np.arccosh(ws))
    alpha = 10*np.log10(1+ee*Cn**2)
    print(f'alpha(n = {n}) = {alpha}')


ee: 0.2589254117941673
alpha(n = 1) = 1.993662702785931
alpha(n = 2) = 6.203272580959268
alpha(n = 3) = 13.418885326100998
alpha(n = 4) = 21.583370328355812
alpha(n = 5) = 29.913681459660868


A partir de $n=4$ se cumple con el requerimiento de atenuación mínima, por lo que nos quedamos con este orden.

La función de aproximación queda:

$|T(j\omega)|^2 = \frac{1}{1+\varepsilon^2.C_4^2}$

Donde $C_4$ tiene la forma:

$C_4 = 8.\omega^4-8.\omega^2+1$

$|T(j\omega)|^2 = \frac{1}{1+\varepsilon^2.(8.\omega^4-8.\omega^2+1)^2}$

$|T(j\omega)|^2 = \frac{1}{1+\varepsilon^2.(64.\omega^8-128.\omega^6+80.\omega^4-16.\omega^2+1)}$

$|T(s)|^2 = \frac{1}{1+\varepsilon^2.(64.s^8+128.s^6+80.s^4+16.s^2+1)}$

$|T(s)|^2 = \frac{1}{\varepsilon^2.64.s^8+\varepsilon^2.128.s^6+\varepsilon^2.80.s^4+\varepsilon^2.16.s^2+(\varepsilon^2+1)}$

Utilizamos python para hallar los polos.

In [3]:
den = [ee*64, ee*0, ee*128, ee*0, ee*80, ee*0, ee*16, ee*0, ee+1]

raices = np.roots(den)

for raiz in raices:
    if np.real(raiz) < 0:
        print(raiz)

(-0.1395359959054355+0.9833791644951989j)
(-0.1395359959054355-0.9833791644951989j)
(-0.3368696937541342+0.4073289868890343j)
(-0.3368696937541342-0.4073289868890343j)


Una vez obtenidos los polos, se puede plantear la transferencia del filtro como 2 secciones de 2do orden:

$T(s) = \frac{1}{s^2+s.0,279+0,9863} . \frac{1}{s^2+s.0,6736+0,2793} . \frac{1}{\sqrt{\varepsilon^2.64}}$

In [4]:
import matplotlib.pyplot as plt
import scipy.signal as sig
from splane import analyze_sys

num = [0.2456]
den = [1, 0.9526, 1.4535, 0.7422, 0.2754]
%matplotlib qt
transfer_f = sig.TransferFunction(num, den)

plt.close('all')
analyze_sys(transfer_f, 'Transferencia')

Estas secciones de 2do orden se pueden implementar con el siguiente circuito:

![circuito](img/ej1_circuito.JPG)

Al simularlo, podemos verificar la transferencia:

![transferencia](img/ej1_transferencia.JPG)

![cursores](img/ej1_cursores.JPG)

Se puede apreciar que cerca de la frecuencia máxima de interés ($10KHz$) tiene una atenuación de prácticamente $1dB$ y para frecuencias mayores a la frecuencia de Nyquist ($15KHz$) la atenuación supera los $20dB$.

In [5]:
# Desnormalización de los componentes

Wz = 1000
Ww = 2*np.pi*10000

C1a_n = 5.068
C1b_n = 0.2995
R1_n = 1
L1_n = 3.5842
C2a_n = 43.21
C2b_n = 2.5539
R2_n = 1
L2_n = 1.4845

C1a_dn = C1a_n / (Ww*Wz)
C1b_dn = C1b_n / (Ww*Wz)
R1_dn = R1_n * Wz
L1_dn = L1_n * Wz / Ww
C2a_dn = C2a_n / (Ww*Wz)
C2b_dn = C2b_n / (Ww*Wz)
R2_dn = R2_n * Wz
L2_dn = L2_n * Wz / Ww

print(f'C1a_dn: {C1a_dn}')
print(f'C1b_dn: {C1b_dn}')
print(f'R1_dn: {R1_dn}')
print(f'L1_dn: {L1_dn}')
print(f'C2a_dn: {C2a_dn}')
print(f'C2b_dn: {C2b_dn}')
print(f'R2_dn: {R2_dn}')
print(f'L2_dn: {L2_dn}')


C1a_dn: 8.065972515897255e-08
C1b_dn: 4.766690545602265e-09
R1_dn: 1000
L1_dn: 0.05704431470299713
C2a_dn: 6.877085091000797e-07
C2b_dn: 4.064658091623915e-08
R2_dn: 1000
L2_dn: 0.023626551301991865


C) Al elegir una frecuencia de sampling 10 veces mayor, nos quedaría que: $f_s=300KHz$ y $f_{nyq} = 150KHz$

Por lo tanto, en la plantilla normalizada, la $\omega_s$ se hallaría en $15$

In [7]:
ws = 15
ee = 10**(1/10)-1
print(f'ee: {ee}')

for n in range(1,4):
    Cn = np.cosh(n*np.arccosh(ws))
    alpha = 10*np.log10(1+ee*Cn**2)
    print(f'alpha(n = {n}) = {alpha}')

ee: 0.2589254117941673
alpha(n = 1) = 17.72748584794907
alpha(n = 2) = 47.176756774247565
alpha(n = 3) = 76.70942089084363


Ahora basta con un orden 2 para cumplir con el requerimiento de $\alpha_{min}$ en $\omega_s$

La función de aproximación queda:

$|T(j\omega)|^2 = \frac{1}{1+\varepsilon^2.C_2^2}$

Donde $C_2$ tiene la forma:

$C_ = 2.\omega^2-1$

$|T(j\omega)|^2 = \frac{1}{1+\varepsilon^2.(2.\omega^2-1)^2}$

$|T(j\omega)|^2 = \frac{1}{1+\varepsilon^2.(4.\omega^4-4.\omega^2+1)}$

$|T(s)|^2 = \frac{1}{1+\varepsilon^2.(4.s^4+4.s^2+1)}$

$|T(s)|^2 = \frac{1}{\varepsilon^2.4.s^4+\varepsilon^2.4.s^2+(1+\varepsilon^2.)}$

Utilizamos Python para hallar los polos:

In [9]:
den = [ee*4, 0, ee*4, 0, (1+ee)]

raices = np.roots(den)

for raiz in raices:
    if np.real(raiz) < 0:
        print(raiz)

(-0.5488671642819637+0.8951285740199127j)
(-0.5488671642819637-0.8951285740199127j)


Una vez obtenidos los polos, se puede plantear la transferencia del filtro:

$T(s) = \frac{1}{s^2+s.1,0976+1.1023}.\frac{1}{\sqrt{\varepsilon^2.4}}$

In [10]:
num = [0.9826]
den = [1, 1.0976, 1.1023]
%matplotlib qt
transfer_f = sig.TransferFunction(num, den)

plt.close('all')
analyze_sys(transfer_f, 'Transferencia')

Este sistema de segundo orden puede implementarse con el siguiente circuito:

![circuito_2](img/ej1_circuito_2.JPG)

Que presenta la siguiente transferencia:

![transferencia_2](img/ej1_transferencia_2.JPG)

![cursores_2](img/ej1_cursores_2.JPG)

Se puede observar que los $20dB$ de atenuación no se encuentran en $f_{nyq}$, pero tampoco están tan cerca de la frecuencia máxima de interés como en el caso anterior.

La principal ventaja de este filtro es que es más simple que el anterior ya que es de menor orden. La desventaja que presenta tiene que ver con que cualquier valor de la señal que exista entre $10KHz$ y $33.65KHz$ donde se encuentra la atenuación de $20dB$ interferirá con nuestra señal ya que no estará del todo atenuada.

In [11]:
# Desnormalización de los componentes

Wz = 1000
Ww = 2*np.pi*10000

R_n = 1
L_n = 0.911
Ca_n = 57.22
Cb_n = 1.0133

R_dn = R_n * Wz
L_dn = (L_n * Wz) / Ww
Ca_dn = Ca_n / (Wz * Ww)
Cb_dn = Cb_n / (Wz * Ww)

print(f'R_dn: {R_dn}')
print(f'L_dn: {L_dn}')
print(f'Ca_dn: {Ca_dn}')
print(f'Cb_dn: {Cb_dn}')

R_dn: 1000
L_dn: 0.014499015315671665
Ca_dn: 9.106845843718251e-07
Cb_dn: 1.6127170383501754e-08
